In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

In [2]:
def get_webpage(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    return bs(page,"html.parser")
    

In [3]:
url = 'https://angkormeas.com/'

In [4]:
soup = get_webpage(url)

In [5]:
find_nav = soup.find('nav')
find_ul = find_nav.find('ul')
find_li = find_ul.find_all('li')

In [6]:
brand_items = []

In [7]:
for li in find_li:
    find_a = li.find_all('a')
    for a in find_a:
        if a.text in ['Computer', 'Smart Phones', 'Tablet|Watch|PC','Accessories']:
#             print(a.text)
            brand_item = {}
            brand_item['brand_name'] = a.text
            brand_item['url'] = a['href']
            brand_items.append(brand_item)

In [24]:
products=[]

In [25]:
for product_url in brand_items:
    product = get_webpage(product_url['url'])
    page_num = product.find('ul',{'class','page-numbers'})
    print(product_url['url'])
    if page_num:
        for index in range(1,len(page_num)):
            test_url = product_url['url']+'page/'+str(index)
            page_url = get_webpage(test_url)
            product_items = page_url.find_all('ul', {'class','products'})
            for product_item in product_items:
                list_items = product_item.find_all('li')

                for item in list_items: 
                    product = {}
                    product['name'] = item.find('h2', {'class', 'woocommerce-loop-product__title'}).text
                    product['link'] = item.find('a')['href']
                    product['image_url'] = item.find('img')['src']

                    price_tag = item.find('span',{'class','price'})
                    if price_tag:
                        if len(price_tag)== 1:
                           normal_price = item.find('span', {'class', 'woocommerce-Price-amount amount'})
                           product['price'] = normal_price.text
                        else:
                           product['price'] = item.find('del').text
                           product['discount'] = item.find('ins').text
                    products.append(product)

https://angkormeas.com/brand/pc/
https://angkormeas.com/product-category/s-phones/
https://angkormeas.com/product-category/tablets/
https://angkormeas.com/product-category/acc/


In [28]:
import json
with open('angkor_meas.json', 'w') as json_file:
    json.dump(products, json_file)